# 목차

어텐션 메커니즘을 사용하여 신경 기계 번역(NMT)을 개선하는 방법에 대한 연구를 다루고 있다. 어텐션은 번역 중에 소스 문장의 특정 부분에 주의를 집중함으로써 성능을 향상시킬 수 있는 메커니즘으로 사용된다. 그러나 어텐션을 활용한 NMT에 적합한 아키텍처에 대한 연구는 아직 부족한 상태

- 글로벌 어텐션 : 모든 원어에 항상 주의를 기울이는 방식
- 로컬 어텐션 : 한 번에 원어의 하위 집합만 살펴보는 방식

이 두 방식의 효과를 영어와 독일어 간의 WMT 번역 작업에서 양방향으로 실험을 통해 입증한다.

### 소개

NMT는 대규모 번역 작업에서 좋은 성능을 가졌다. 예를 들어 eng -> fr , eng -> de 번역 작업에 좋은 결과 출력했다. NMT는 최소한의 도메인 지식이 필요해 개념적으로 단순하다는 장점이 있다.




# 1. 소개

- Luong et al. (2015)의 모델은 모든 소스 단어를 읽고 문장 끝의 심볼 **eos** 에 도달하면 타겟 단어를 하나씩 생성
- 모델은 장문의 단어 시퀀스에 잘 일반화되고, 기존의 MT와 달리 거대한 어구 테이블이나 언어 모델을 명시적으로 저장할 필요가 없어 메모리 효율이 높다.
- NMT 디코더의 구현이 간단하며, 일반적인 MT에서 복잡한 디코더와 달리 쉽게 이루어짐

### Attention

- 다양한 모달 간의 정렬을 학습하게 함으로써, 예를 들어 이미지와 동적 제어 문제에서의 객체 및 에이전트 행동 간, 음성 인식 작업에서 음성 프레임과 텍스트 간, 그림과 텍스트 설명 간 등 다양한 모델 간의 정렬을 학습할 수 있다.

#### 연구 목적 및 설계

현재까지 NMT에서 어텐션을 사용한 아키텍처에 대한 탐색이 부족한 상태입니다.
본 연구에서는 간결하면서 효과적인 두 가지 어텐션 기반 모델을 디자인했습니다.
하나는 모든 소스 단어에 항상 주의를 기울이는 글로벌 어텐션 방식이고, 다른 하나는 시간당 일부 소스 단어에만 주의를 기울이는 로컬 어텐션 방식입니다.

#### 결과 및 분석

실험 결과로는 로컬 어텐션을 사용하면 기존의 어텐션 없는 시스템 대비 5.0 BLEU 포인트의 성능 향상이 있었습니다.
영어에서 독일어로의 번역에서는 WMT'14 및 WMT'15에서 이전 SOTA 시스템보다 1.0 BLEU 포인트 이상의 성능 향상이 있었습니다.
학습, 장문 처리 능력, 어텐션 아키텍처 선택, 정렬 품질, 번역 결과 등 다양한 측면에서 모델을 평가하기 위해 광범위한 분석을 수행했습니다.

# 2. 신경 기계 번역(NMT) 소개
- NMT는 소스 문장 x1, ..., xn을 타겟 문장 y1, ..., ym으로 번역하는 조건부 확률 p(y|x)를 직접 모델링하는 신경망입니다.
- 기본적인 NMT는 (a) 각 소스 문장에 대한 표현 s를 계산하는 인코더와 (b) 하나의 타겟 단어를 생성하고 조건부 확률을 분해하는 디코더로 구성됩니다.
### NMT 모델 구성

- 조건부 확률을 디코더에서 분해하기 위해 일반적으로 순환 신경망(RNN) 아키텍처를 사용합니다.
- RNN은 최근 NMT 작업에서 흔히 사용되며, 그러나 디코더 및 인코더에서 어떤 RNN 아키텍처를 사용하고 인코더가 소스 문장 표현을 계산하는 방법에 차이가 있습니다.

### 디코더의 단어 확률 모델

- 각 단어 yj의 디코딩 확률을 매개변수화할 수 있습니다.
- 확률 p(yj|y<j, s)은 softmax(g(hj))로 계산되며, 여기서 g는 어휘 크기의 벡터를 출력하는 변환 함수입니다.
- hj는 RNN 숨겨진 유닛으로 이전 숨겨진 상태에 따라 계산되며, f(hj-1, s)로 나타낼 수 있습니다. f는 현재 숨겨진 상태를 이전 숨겨진 상태를 기반으로 계산하며, vanilla RNN 유닛, GRU 또는 LSTM 유닛이 될 수 있습니다.

### 어텐션 메커니즘 도입

- 어텐션 메커니즘은 전체 번역 과정 동안 상태를 유지하면서 소스 문장 표현을 사용하는 방식입니다.
- 이 작업에서는 Sutskever et al. (2014)와 Luong et al. (2015)을 따라 스태킹 LSTM 아키텍처를 사용하며, 이를 통해 어텐션 메커니즘을 도입합니다.
- 학습 목적

- 훈련 목적은 주어진 병렬 훈련 말뭉치 D에 대해 로그 확률을 최소화하는 것입니다.
- 목적 함수 Jt = ∑(-log p(y|x))는 각 훈련 샘플 (x, y)에 대한 로그 확률을 합산한 것입니다.

# 3. 어텐션(Attention) 모델
글로벌 어텐션 및 로컬 어텐션 두 모델 타입을 각각 Figure 2 및 Figure 3에 설명되어 있는 대로 나타냄


### 모델 공통점

- 이 두 유형의 모델에는 각 디코딩 단계에서 공통적으로 적용되는 특징이 있습니다.
- 각 단계에서 먼저 스태킹 LSTM의 최상위 레이어의 숨겨진 상태 ht를 입력으로 취합니다.
- 그런 다음 목표는 현재 타겟 단어 yt을 예측하는 데 도움이 되는 관련 소스 측 정보를 포함하는 컨텍스트 벡터 ct를 유도하는 것입니다.
- 컨텍스트 벡터 ct의 유래 방식은 모델 유형에 따라 다르지만, 이후 단계에서는 동일한 절차를 공유합니다.

### 어텐션 메커니즘

- 주어진 타겟 숨겨진 상태 ht와 소스 측 컨텍스트 벡터 ct를 사용하여 두 벡터에서 정보를 결합하기 위해 간단한 연결 레이어를 사용합니다.
- 이를 통해 어텐션(hidden) 상태 h˜t를 생성합니다: h˜t = tanh(Wc[ct; ht]) (식 5).
- 어텐션 벡터 h˜t는 소프트맥스 레이어를 통과하여 확률 분포를 생성합니다: p(yt|y<t, x) = softmax(Wsh˜t) (식 6).

### 3-1.Global Attention(전역 어텐션)
